### Reload .py files whenever there are changes

In [269]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Local imports from this project

In [270]:
from mdf import *
from mdf.ops import *
from mdf.data import *

### External imports from dependencies

In [271]:
from pytorch3d.ops.laplacian_matrices import norm_laplacian, cot_laplacian, laplacian
from pytorch3d.ops import norm_laplacian, cot_laplacian, laplacian
from pyvista.plotting.plotter import Plotter
import torchvision.transforms.v2 as T
import torch.distributions as distrib
import plotly.graph_objects as go
import plotly.express as px
from torch import Tensor
from typing import cast
import torch_geometric
import torch.nn as nn
import pyvista as pv
import scipy.linalg
import torch.linalg
import numpy as np
import pytorch3d
import einops
import torch

### Environment setup

In [272]:
pv.set_jupyter_backend('trame')

### Data Loading

In [273]:
# Download standard shapes
data = DataManager(data_dir=data_dir, cache_dir=cache_dir)

### How to apply a texture to a 3d mesh

![https://github.com/pyvista/pyvista-support/issues/168](https://user-images.githubusercontent.com/22067021/83365096-c2069800-a373-11ea-9cf3-418f9e55147c.png)

In [274]:
p = Plotter()
manifold  = cast(pv.PolyData, data.objects['stanford-bunny'].vista)
texture = pv.Texture(data.weather[3600]['data'])
manifold.texture_map_to_sphere(inplace=True)
p.add_mesh(manifold, texture=texture)
p.camera.tight()
p.show()

/home/invokariman/.cache/pypoetry/virtualenvs/mdf-B-9uJKhD-py3.11/lib/python3.11/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x735644898790_193&reconnect=auto" class="py…

### Compute Positional Embedding using Laplace-Beltrami Operator

In [275]:
lbo = LBOEmbedding.from_poly(manifold, k_evecs=100, device=torch.device('cuda'))

In [276]:
# Create 3d mesh
mesh = pv.PolyData(lbo.verts.numpy(), lbo.faces.numpy())
evecs = lbo(torch.arange(lbo.verts.size(0)))

# Plot multiple images
p = Plotter(shape=(1, 3))

# 1st eigenfunction
p.subplot(0, 0)
p.add_mesh(mesh.copy(), scalars=evecs[:, 1:2].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 2nd eigenfunction
p.subplot(0, 1)
mesh = mesh.copy()
p.add_mesh(mesh.copy(), scalars=evecs[:, 50:51].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 3rd eigenfunction
p.subplot(0, 2)
mesh = mesh.copy()
p.add_mesh(mesh, scalars=evecs[:, 99:100].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

p.show()

Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x735624be1950_194&reconnect=auto" class="py…

In [277]:
p = Plotter()

# Create 3d mesh
m_sampler = MeshSampler(verts=lbo.verts, faces=lbo.faces)
pts, verts, coefs = m_sampler.sample(2500)
evecs = lbo(verts, coefs)
mesh = pv.PolyData(pts.numpy())

# Plot multiple images
p = Plotter(shape=(1, 3))

# 1st eigenfunction
p.subplot(0, 0)
p.add_mesh(mesh, scalars=evecs[:, 1:2].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 2nd eigenfunction
p.subplot(0, 1)
mesh = mesh.copy()
p.add_mesh(mesh.copy(), scalars=evecs[:, 50:51].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

# 3rd eigenfunction
p.subplot(0, 2)
mesh = mesh.copy()
p.add_mesh(mesh, scalars=evecs[:, 75:76].sum(dim=1), clim=[-1, 1], cmap='RdBu_r')
p.camera.tight()

p.show()

Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x735624a2f510_196&reconnect=auto" class="py…

### Compute texture using implicit UV mapping

In [278]:
image = data.weather[3600]['data']
mesh.texture_map_to_sphere(inplace=True)
texture = pv.Texture(image)

p = Plotter()
p.add_mesh(mesh, texture=texture)
p.camera.tight()
p.show()

Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x73563f0ced50_197&reconnect=auto" class="py…

### Compute texture using f : M -> Y interpolation

In [283]:
# Function that maps from M to Y
mesh2rgb = TextureMapper()
rgb = mesh2rgb(pts, torch.tensor(image))
mesh['colors'] = rgb

p = Plotter()
p.add_mesh(mesh, scalars='colors', rgb=True)
p.camera.tight()
p.show()

Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x735641a66110_200&reconnect=auto" class="py…

### Place signal Y on the initial mesh without texture and perform interpolation

In [280]:
p = Plotter()
p.add_mesh(mesh, scalars='colors', rgb=True)
p.camera.tight()
p.show()

Widget(value='<iframe src="http://localhost:46281/index.html?ui=P_0x73562a484050_199&reconnect=auto" class="py…